In [1]:
import tensorflow as tf
import os
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline


In [27]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.applications import inception_resnet_v2
#from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop

from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Dropout, Flatten, Input, Dense
from keras import regularizers
from keras import initializers

In [4]:
path = '/home/jgomez/Documents/Us'

In [5]:
train_dir = path + "/train"
test_dir = path + "/val"

In [6]:
datagen_train = ImageDataGenerator(
    rescale=1./255,
      rotation_range=180,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.1,
      zoom_range=[0.9, 1.5],
      horizontal_flip=True,
      vertical_flip=True,
      fill_mode='nearest')

In [28]:

def create_model():
    
    def add_conv_block(model, num_filters):
        
        model.add(Conv2D(num_filters, 3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(l = 0.01), kernel_initializer = 'he_normal' ))
        model.add(MaxPooling2D(pool_size=2))
        model.add(Conv2D(num_filters, 3, activation='relu', padding='valid', kernel_regularizer=regularizers.l2(l = 0.01), kernel_initializer = 'he_normal'))
        model.add(MaxPooling2D(pool_size=2))
        model.add(Conv2D(num_filters, 3, activation='relu', padding='valid', kernel_regularizer=regularizers.l2(l = 0.01), kernel_initializer = 'he_normal'))
        model.add(BatchNormalization())
        model.add(Conv2D(num_filters, 3, activation='relu', padding='valid', kernel_regularizer=regularizers.l2(l = 0.01), kernel_initializer = 'he_normal'))
        model.add(BatchNormalization())
        model.add(Conv2D(num_filters, 3, activation='relu', padding='valid', kernel_regularizer=regularizers.l2(l = 0.01), kernel_initializer = 'he_normal'))
        model.add(MaxPooling2D(pool_size=2))
        model.add(BatchNormalization())

        return model
    
    model = tf.keras.models.Sequential()
    model.add(Input(shape=(224, 224, 3)))
    
    model = add_conv_block(model, 32)
    model.add(Flatten())
    model.add(Dense(2))
    model.add(Dense(2, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = create_model()
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 110, 110, 32)      9248      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 55, 55, 32)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 53, 53, 32)        9248      
_________________________________________________________________
batch_normalization_12 (Batc (None, 53, 53, 32)        128       
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 51, 51, 32)       

In [29]:
batch_size = 32

In [30]:
input_shape = model.layers[0].output_shape[1:3]
input_shape

(224, 224)

In [14]:
generator_train = datagen_train.flow_from_directory(directory=train_dir,
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    shuffle=True)

Found 45646 images belonging to 2 classes.


In [15]:
datagen_test = ImageDataGenerator(rescale=1./255)

In [16]:
generator_test = datagen_test.flow_from_directory(directory=test_dir,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  shuffle=False)

Found 1194 images belonging to 2 classes.


In [ ]:
h = model.fit(
    generator_train,
    validation_data=generator_test,
    epochs=50, 
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=15),
        tf.keras.callbacks.ModelCheckpoint('models/model_{val_accuracy:.3f}.h5', save_best_only=True,
                                          save_weights_only=False, monitor='val_accuracy')
    ]
)

Epoch 1/50
1427/1427 [==============================] - 1085s 760ms/step - loss: 2.0783 - accuracy: 0.5087 - val_loss: 0.9330 - val_accuracy: 0.5034
Epoch 2/50
 244/1427 [====>.........................] - ETA: 14:42 - loss: 0.8760 - accuracy: 0.5184